### 项目说明

在搜索竞价广告系统中，用户通过在搜索引擎输入具体的查询词来获取相关信息。因此，用户的历史查询词与用户的基本属性及潜在需求有密切的关系。

举例如下：

    年龄在19岁至23岁区间的自然人会有较多的搜索行为与大学生活、社交等主题有关

    男性相比女性会在军事、汽车等主题有更多的搜索行为

    高学历人群会更加倾向于获取社会、经济等主题的信息

根据用户历史一个月的查询记录，以用户的人口属性（包括性别、年龄、学历）为标签，通过机器学习、数据挖掘技术构建分类算法来对新增用户的人口属性进行判定。

### 数据说明
字段	说明

    ID	加密后的ID
    Age	0：未知年龄; 1：0-18岁; 2：19-23岁; 3：24-30岁; 4：31-40岁; 5：41-50岁; 6： 51-999岁
    Gender	0：未知  1：男性  2：女性
    Education	0：未知学历; 1：博士; 2：硕士; 3：大学生; 4：高中; 5：初中; 6：小学
    Query List	搜索词列表

### 导包

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
%matplotlib inline

### 数据加载

#### 验证编码格式

In [38]:
def read_file(file_path,mode='rb'):
    with open(file_path,mode=mode) as f:
        i = 0
        while True:
            line = f.readline()
            i += 1
            yield line
train_file = read_file('../../dataset/personas/user_tag_query.10W.TRAIN.csv')
test_file = read_file('../../dataset/personas/user_tag_query.10W.TEST.csv')

In [39]:
import chardet
print(chardet.detect(next(train_file)))
print(chardet.detect(next(test_file)))

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'GB2312', 'confidence': 0.99, 'language': 'Chinese'}


In [40]:
import chardet
print(chardet.detect(next(train_file)))
print(chardet.detect(next(test_file)))

{'encoding': 'GB2312', 'confidence': 0.99, 'language': 'Chinese'}
{'encoding': 'GB2312', 'confidence': 0.99, 'language': 'Chinese'}


#### 观察文件格式

In [41]:
next(train_file).decode('GB2312')[:500]

'43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只是不想让你支付,原谅我的无情,对不起\t处女座代表的花朵\t烤鸡胸肉的做法 烤箱\t不曾忘记,也不曾想起\t一辈子那么长 一天没走到终点\t联塑pvc排水管规格表\t大王椰\t性格文静是什么意思\t250ml牛奶用多少克奶粉冲\t化蝶去寻花 夜夜栖芳草什么意思\t学会爱自己 才会真正懂得爱1001学会爱自己 \t福睿斯\t斗鱼tv\t厨宝是什么\t厨宝烤箱\t禹州城市广场电影院\t大王棕树图片\t王棕树图片\t发酵箱\t6920882798458\t我只是不想让你支付,原谅我的无情,对不起10\t你住的城市下雨了,很想问你有没有带伞 可是\t250ml牛奶用多少克奶粉\t烤翅的做法烤箱\t全脂奶粉泡的比例\t牛肉怎么炒才嫩\t厨宝牌是谁的\t猴配猴的婚姻怎么样\t可惜没如果mv爱情公寓\t守不住的才是个笑话\t犯太岁是什么意思\t禹州城市广场电影院1001禹州城市广场电影院\t烤箱什么牌子好\t全脂奶粉怎么泡\t红铁人面粉蛋白质含量\t你走后我却活成你的样子\t鸡排的做法大全\t放爱\t斗鱼直播\t'

In [42]:
next(test_file).decode('GB2312')[:500]

'CA9F675A024FB2353849350A35CF8B0F\t黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\t重庆重钢老板\t资阳俊士\t2016lpl夏季赛积分\t主角修炼无情道的小说\t重庆重钢\t风流小农民艳遇记\t主角以无情杀戮修炼\t网店推广\t2016lpl夏季赛omg是否保级成功\t成都工商管理学院学费多少 小说\t手机上怎么扫描二维码\t全能运动员\t男主很阴暗残忍的小说\tvertu手机\t胸型号\t中英文在线翻译\t神秘的性奖励班会小说\t成都新都龙桥地图\t薛之谦张婉清\t网店的开张以及宣传 小说\t重钢亏六十亿\t英雄联盟之观战系统\t人民公敌\t黑暗文小说\t主角修魔杀戮小说\t成都旅游经典\t史上第一大魔神\t价格超过十万的手机\twww6090青苹果影院\t周星驰多少岁出名\t薛之谦\t2016八月中国经济比去年增长多少\t中英文翻译 小说\t阿斯顿马丁\t亵渎\t主角杀戮无女主小说\tgood luck什么意思\tcm朋克ufc首秀\t新都龙桥自行车\t演员薛之谦张婉清\t重庆重钢集团资产多少\t男主角性格阴暗的小说\t隐藏分查询\t最强特种兵之王\t中国迪士尼乐园在哪个城市\t主角是反派无女主小说\t野'

#### 文件格式化

train_data 是以'，'分隔的标准csv文件

test_data 是以空格分开的csv文件，同时QueryList字段内的搜索字段也是以空格分开，故只能按空格分割一次

##### 预测集格式化

In [8]:
import chardet
import csv
def load_file(file_path):
    with open(file_path,encoding='GB2312',errors='ignore') as f:
        while True:
                line = f.readline()
                yield line
                if not line:
                    print('文件加载结束')
                    break
def format_and_save_file(file_text, save_file_path,columns=None):       
    csv_writer = csv.writer(open(save_file_path, 'w',encoding='utf-8',newline=''))
    if columns:
        csv_writer.writerow(columns)
    while True:
        try:
            csv_writer.writerow(next(file_text).replace('\n','').split('\t',maxsplit=1))
        except Exception as e:
            print(e)
            print('文件格式化结束')
            break
test_file_text = load_file('../../dataset/personas/user_tag_query.10W.TEST.csv')
format_and_save_file(test_file_text, '../../dataset/personas/utf_8_user_tag_query.10W.TEST.csv',['ID','QueryList'])

文件加载结束

文件格式化结束


##### 训练集格式化

In [43]:
train_file_data  = pd.read_csv('../../dataset/personas/user_tag_query.10W.TRAIN.csv', encoding='gb18030')
train_file_data.to_csv('../../dataset/personas/utf_8_user_tag_query.10W.TRAIN.csv', encoding='utf-8', index=False)

#### 文件加载

In [44]:
train_data  = pd.read_csv('../../dataset/personas/utf_8_user_tag_query.10W.TRAIN.csv')

In [45]:
test_data  = pd.read_csv('../../dataset/personas/utf_8_user_tag_query.10W.TEST.csv')

### 数据集成

In [46]:
train_data.columns

Index(['ID', 'age', 'Gender', 'Education', 'QueryList'], dtype='object')

In [47]:
train_data.head()

,ID,age,Gender,Education,QueryList
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [48]:
test_data.columns

Index(['ID', 'QueryList'], dtype='object')

In [49]:
data = pd.concat([train_data.reindex(columns=['ID', 'QueryList','age', 'Gender', 'Education']),test_data],sort=False,ignore_index=True)

In [50]:
data.shape

(199832, 5)

In [51]:
data.head()

,ID,QueryList,age,Gender,Education
0,22DD920316420BE2DF8D6EE651BA174B,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,1.0,1.0,4.0
1,43CC3AF5A8D6430A3B572337A889AFE4,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",2.0,1.0,3.0
2,E97654BFF5570E2CCD433EA6128EAC19,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...,4.0,1.0,0.0
3,6931EFC26D229CCFCEA125D3F3C21E57,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,4.0,2.0,3.0
4,E780470C3BB0D340334BD08CDCC3C71A,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,2.0,2.0,4.0


In [52]:
data.tail()

,ID,QueryList,age,Gender,Education
199827,4AB983FE74DCB5B04FA0A8CE7779E2EC,东北一家人沈腾\t叶小白有关的小说\t姚启圣\t李小狼\t七煌老板孙博文\t珠海鸿景花园\t...,NaN,NaN,NaN
199828,8FCE58D7DA890DF4F6365283E02F936D,沈阳天士力药房\t儿童支气管炎\t肺炎10天点滴还咳嗽\t熊岳虹吸谷\t过敏性咳嗽\t饭团子...,NaN,NaN,NaN
199829,0821784C7EFD4FC3C96FE8EE52989551,乡村小神医\t脸上各种斑图片及名称\t猕猴桃是热性还是凉性\t梦见外公又死了\t离婚了你还爱...,NaN,NaN,NaN
199830,BF98531D782D4C31CC26202081E71E4B,经营策略分析\t经营组织论第一节\t腾讯微博\t盛世光年婚礼视频优酷\ttopik等级划分\...,NaN,NaN,NaN
199831,NaN,NaN,NaN,NaN,NaN


In [53]:
data.drop(199831, inplace=True)
data.tail()

,ID,QueryList,age,Gender,Education
199826,B4D8E2DA560327C4D6F5D66CEB451209,安乃近\t大太平\t温秀 造句\t似乎造句\t若是\t无声之手\t洋葱\t不过\t清吉太平\...,NaN,NaN,NaN
199827,4AB983FE74DCB5B04FA0A8CE7779E2EC,东北一家人沈腾\t叶小白有关的小说\t姚启圣\t李小狼\t七煌老板孙博文\t珠海鸿景花园\t...,NaN,NaN,NaN
199828,8FCE58D7DA890DF4F6365283E02F936D,沈阳天士力药房\t儿童支气管炎\t肺炎10天点滴还咳嗽\t熊岳虹吸谷\t过敏性咳嗽\t饭团子...,NaN,NaN,NaN
199829,0821784C7EFD4FC3C96FE8EE52989551,乡村小神医\t脸上各种斑图片及名称\t猕猴桃是热性还是凉性\t梦见外公又死了\t离婚了你还爱...,NaN,NaN,NaN
199830,BF98531D782D4C31CC26202081E71E4B,经营策略分析\t经营组织论第一节\t腾讯微博\t盛世光年婚礼视频优酷\ttopik等级划分\...,NaN,NaN,NaN


### 数据观察

#### 数据规模

In [54]:
data.shape

(199831, 5)

In [55]:
train_data.shape

(99831, 5)

In [56]:
test_data.shape

(100001, 2)

#### 数据字段

In [57]:
train_data.columns

Index(['ID', 'age', 'Gender', 'Education', 'QueryList'], dtype='object')

In [58]:
test_data.columns

Index(['ID', 'QueryList'], dtype='object')

In [59]:
data.columns

Index(['ID', 'QueryList', 'age', 'Gender', 'Education'], dtype='object')

#### 数据描述性统计

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199831 entries, 0 to 199830
Data columns (total 5 columns):
ID           199831 non-null object
QueryList    199831 non-null object
age          99831 non-null float64
Gender       99831 non-null float64
Education    99831 non-null float64
dtypes: float64(3), object(2)
memory usage: 9.1+ MB


In [61]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99831 entries, 0 to 99830
Data columns (total 5 columns):
ID           99831 non-null object
age          99831 non-null int64
Gender       99831 non-null int64
Education    99831 non-null int64
QueryList    99831 non-null object
dtypes: int64(3), object(2)
memory usage: 3.8+ MB


In [62]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 0 to 100000
Data columns (total 2 columns):
ID           100000 non-null object
QueryList    100000 non-null object
dtypes: object(2)
memory usage: 1.5+ MB


In [63]:
train_data.describe()

,age,Gender,Education
count,99831.000000,99831.000000,99831.000000
mean,2.082089,1.387184,3.903547
std,1.184173,0.529536,1.521979
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,3.000000
50%,2.000000,1.000000,4.000000
75%,3.000000,2.000000,5.000000
max,6.000000,2.000000,6.000000


#### 数据前后几条信息

In [64]:
data.head()

,ID,QueryList,age,Gender,Education
0,22DD920316420BE2DF8D6EE651BA174B,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,1.0,1.0,4.0
1,43CC3AF5A8D6430A3B572337A889AFE4,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",2.0,1.0,3.0
2,E97654BFF5570E2CCD433EA6128EAC19,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...,4.0,1.0,0.0
3,6931EFC26D229CCFCEA125D3F3C21E57,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,4.0,2.0,3.0
4,E780470C3BB0D340334BD08CDCC3C71A,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,2.0,2.0,4.0


In [65]:
data.tail()

,ID,QueryList,age,Gender,Education
199826,B4D8E2DA560327C4D6F5D66CEB451209,安乃近\t大太平\t温秀 造句\t似乎造句\t若是\t无声之手\t洋葱\t不过\t清吉太平\...,NaN,NaN,NaN
199827,4AB983FE74DCB5B04FA0A8CE7779E2EC,东北一家人沈腾\t叶小白有关的小说\t姚启圣\t李小狼\t七煌老板孙博文\t珠海鸿景花园\t...,NaN,NaN,NaN
199828,8FCE58D7DA890DF4F6365283E02F936D,沈阳天士力药房\t儿童支气管炎\t肺炎10天点滴还咳嗽\t熊岳虹吸谷\t过敏性咳嗽\t饭团子...,NaN,NaN,NaN
199829,0821784C7EFD4FC3C96FE8EE52989551,乡村小神医\t脸上各种斑图片及名称\t猕猴桃是热性还是凉性\t梦见外公又死了\t离婚了你还爱...,NaN,NaN,NaN
199830,BF98531D782D4C31CC26202081E71E4B,经营策略分析\t经营组织论第一节\t腾讯微博\t盛世光年婚礼视频优酷\ttopik等级划分\...,NaN,NaN,NaN


In [66]:
train_data.head()

,ID,age,Gender,Education,QueryList
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [28]:
test_data.head()

,ID,QueryList
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...


### 语料库准备

#### 停用词加载

In [15]:
stopwords_list = []
with open('../../common/stopwords1893.txt', 'r', encoding='utf-8') as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords_list.append(line.replace('\n', ''))

In [16]:
stopwords_list[:5]

['!', '"', '#', '$', '%']

#### 分词、停用词去除、词性过滤

In [24]:
# 根据词性过滤
import jieba
import jieba.posseg

# 添加自定义词语
jieba.add_word('王者荣耀',tag='n')
jieba.add_word('百度云',tag='n')
jieba.add_word('徽信',tag='n')
jieba.add_word('电子发票',tag='n')
jieba.add_word( '表情包',tag='n')
jieba.add_word( '萌妻',tag='n')
jieba.add_word('水煮',tag='v')

posseg_corpus = []
i = 0
rows = float(data.QueryList.shape[0]) /100
def tokenize_posseg(sentences):
    global i
    cleaned_sentences = sentences.replace('\t', ' ')
    token = jieba.posseg.cut(sentences)
    flag_list = ['n', 'nz', 'nr','ns', 'nt','ng', 'v','vn', 'j', 'i']
    word_list = [word.word for word in list(token) 
                 if word.word not in stopwords_list 
                 and word.flag in flag_list
                ]
    if not word_list:
        posseg_corpus.append(' '.join([word.word for word in list(token) 
                 if word.word not in stopwords_list 
                ]))
    else:
        posseg_corpus.append(' '.join(word_list))
    i +=1
#     if i % 100 == 0:
    print('\r 进度：%0.4f%%' % (i/rows), end="")
    return ' '.join(word_list)
data['PossegCleanedQueryList'] = data.QueryList.apply(tokenize_posseg)

 进度：100.0000%

In [69]:
data['PossegCleanedQueryList'] = [posseg_corpu for posseg_corpu in posseg_corpus]

In [70]:
data.PossegCleanedQueryList.head()

0    双沟 女生 中财网 财经 周公 解梦 查询 曹云金 郭德纲 总裁 大人 行行好 中财网 财经...
1    广州 厨宝 烤箱 世情 人情 雨 送 花易落 晓 风干 泪痕 厦门 酒店用品 批发市场 不想...
2    钻石 泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 盘锦 台安 网游 网游 辽 北镇 ...
3    受欢迎 狗狗 排行榜 舶 读 场景 描 写 范例 绘图 软件 枣 酸奶 吃 租 衣服 网站 ...
4    干槽症 太太 叶 没去 美国 干槽症 眼皮 跳 麦 旋风 勺子 吉林市 鹿王 制药 股份 有...
Name: PossegCleanedQueryList, dtype: object

#### 语料库数据持久化

In [30]:
# 经过词性过滤
posseg_save_file_path = '../../dataset/personas/posseg_word_corpus.text'
with open(posseg_save_file_path, 'w', encoding='utf-8') as f:
    for posseg_corpu in posseg_corpus:
        if isinstance(posseg_corpu,list):
            posseg_corpu = ' '.join(posseg_corpu)
        f.write(posseg_corpu)
        f.write('\n')

#### 结构化数据查看

In [72]:
posseg_cleaned_query_list = pd.read_csv('../../dataset/personas/posseg_word_corpus.text',header=None, names=['posseg_cleaned_query_list'])

In [73]:
posseg_cleaned_query_list.head()

,posseg_cleaned_query_list
0,双沟 女生 中财网 财经 周公 解梦 查询 曹云金 郭德纲 总裁 大人 行行好 中财网 财经...
1,广州 厨宝 烤箱 世情 人情 雨 送 花易落 晓 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,钻石 泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 盘锦 台安 网游 网游 辽 北镇 ...
3,受欢迎 狗狗 排行榜 舶 读 场景 描 写 范例 绘图 软件 枣 酸奶 吃 租 衣服 网站 ...
4,干槽症 太太 叶 没去 美国 干槽症 眼皮 跳 麦 旋风 勺子 吉林市 鹿王 制药 股份 有...


In [75]:
posseg_corpus[0][:100]

'双沟 女生 中财网 财经 周公 解梦 查询 曹云金 郭德纲 总裁 大人 行行好 中财网 财经 传媒 教师节 全文 男子 砸毁 墓碑 黄岩岛 填 海图 缘 落跑 甜心 梁朝伟 替身 框 笑傲江湖 电视剧'

In [78]:
posseg_cleaned_query_list.tail()

,posseg_cleaned_query_list
199822,安乃近 太平 温秀 造句 造句 手 洋葱 清吉 太平 笙箫 电视剧 倾心 心 倾城 吃 西药...
199823,东北 一家人 沈腾 叶小白 小说 姚启圣 李小狼 七煌 老板 博文 珠海 鸿景 花园 云菲菲...
199824,沈阳 天士力 药房 儿童 支气管炎 肺炎 天 点滴 咳嗽 熊岳 谷 过敏性 咳嗽 饭团子 饭...
199825,乡村 神医 斑 图片 名称 猕猴桃 热性 凉性 梦见 外公 死 离婚 爱我吗 女孩子 发型 ...
199826,经营策略 分析 经营 组织 腾讯 博 盛世 光年 婚礼 视频 划分 临摹 照片 济南 恒隆 ...


In [79]:
data['PossegCleanedQueryList'].shape

(199831,)

In [80]:
data.head()

,ID,QueryList,age,Gender,Education,PossegCleanedQueryList
0,22DD920316420BE2DF8D6EE651BA174B,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,1.0,1.0,4.0,双沟 女生 中财网 财经 周公 解梦 查询 曹云金 郭德纲 总裁 大人 行行好 中财网 财经...
1,43CC3AF5A8D6430A3B572337A889AFE4,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",2.0,1.0,3.0,广州 厨宝 烤箱 世情 人情 雨 送 花易落 晓 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,E97654BFF5570E2CCD433EA6128EAC19,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...,4.0,1.0,0.0,钻石 泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 盘锦 台安 网游 网游 辽 北镇 ...
3,6931EFC26D229CCFCEA125D3F3C21E57,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,4.0,2.0,3.0,受欢迎 狗狗 排行榜 舶 读 场景 描 写 范例 绘图 软件 枣 酸奶 吃 租 衣服 网站 ...
4,E780470C3BB0D340334BD08CDCC3C71A,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,2.0,2.0,4.0,干槽症 太太 叶 没去 美国 干槽症 眼皮 跳 麦 旋风 勺子 吉林市 鹿王 制药 股份 有...


In [82]:
data.PossegCleanedQueryList.head()

0    双沟 女生 中财网 财经 周公 解梦 查询 曹云金 郭德纲 总裁 大人 行行好 中财网 财经...
1    广州 厨宝 烤箱 世情 人情 雨 送 花易落 晓 风干 泪痕 厦门 酒店用品 批发市场 不想...
2    钻石 泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 盘锦 台安 网游 网游 辽 北镇 ...
3    受欢迎 狗狗 排行榜 舶 读 场景 描 写 范例 绘图 软件 枣 酸奶 吃 租 衣服 网站 ...
4    干槽症 太太 叶 没去 美国 干槽症 眼皮 跳 麦 旋风 勺子 吉林市 鹿王 制药 股份 有...
Name: PossegCleanedQueryList, dtype: object

In [83]:
data.PossegCleanedQueryList.shape

(199831,)

### word2vec词向量模型

#### 模型训练

In [95]:
import jieba 
corpus = []
lenth = len(posseg_corpus)
i = 0
for corpu in posseg_corpus:
    if not isinstance(corpu, list):
        corpus.append(list(jieba.cut(corpu)))
    else:
        corpus.append(corpu)
    i += 1
    print('\r 进度：%0.4f%%' % (i/lenth*100), end="")


 进度：1.0000%

In [96]:
from gensim.models import Word2Vec
word2vec_posseg_model = Word2Vec(corpus, size=300, min_count=1)

In [97]:
word2vec_posseg_model.corpus_count

199831

In [98]:
len(word2vec_posseg_model.wv.vocab)

752385

In [99]:
word2vec_posseg_model.wv.most_similar('北京')

C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('北京市', 0.6917001008987427),
 ('昌平', 0.6040279865264893),
 ('天津', 0.6036925315856934),
 ('张家口', 0.5988921523094177),
 ('顺义', 0.5951307415962219),
 ('怀柔', 0.5865505933761597),
 ('丰台', 0.5807821750640869),
 ('燕郊', 0.5797988176345825),
 ('保定', 0.5649673938751221),
 ('石家庄', 0.5543346405029297)]

In [100]:
word2vec_posseg_model.wv.most_similar('房子')

[('楼房', 0.7589250802993774),
 ('新房', 0.7165311574935913),
 ('房屋', 0.7006446123123169),
 ('毛坯房', 0.6985774040222168),
 ('老房子', 0.6930108070373535),
 ('新房子', 0.6796026825904846),
 ('房能', 0.6727418303489685),
 ('买房子', 0.6633325815200806),
 ('大房子', 0.6312969923019409),
 ('商品房', 0.6223334074020386)]

In [101]:
word2vec_posseg_model.wv.similarity('兄弟','姐妹')

0.29962897

In [102]:
word2vec_posseg_model.wv.similarity('房子','车子')

0.33978018

In [103]:
word2vec_posseg_model.wv.similarity('老婆','孩子')

0.244461

#### 模型保存

In [104]:
word2vec_posseg_model.save('../../models/personas_word2vec_posseg.model')

#### 模型加载

In [105]:
from gensim.models import Word2Vec
word2vec_posseg_model = Word2Vec.load('../../models/personas_word2vec_posseg.model')

In [107]:
word2vec_posseg_model.wv.most_similar('北京')

C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('北京市', 0.6917001008987427),
 ('昌平', 0.6040279865264893),
 ('天津', 0.6036925315856934),
 ('张家口', 0.5988921523094177),
 ('顺义', 0.5951307415962219),
 ('怀柔', 0.5865505933761597),
 ('丰台', 0.5807821750640869),
 ('燕郊', 0.5797988176345825),
 ('保定', 0.5649673938751221),
 ('石家庄', 0.5543346405029297)]

### 搜索结果平均向量化

#### 求平均向量
其目的在于构造每个用户搜索特征向量

In [108]:
posseg_file_name = '../../dataset/personas/posseg_word_corpus.text'
with open(posseg_file_name, 'r',encoding='utf-8') as f:
    index = 0
    lines = f.readlines()
    posseg_doc_vec = np.zeros((len(lines),300))
    for line in lines:
        word_vec = np.zeros((1,300))
        wrod_num = 0
        for word in line:
            if word in word2vec_posseg_model:
                wrod_num += 1
                word_vec += np.array([word2vec_posseg_model[word]])
        if sum(np.isnan(word_vec)).sum():
            print(line)
            print(index)
        print('\r [index]:%d' % index, end='')
        posseg_doc_vec[index] = word_vec / float(wrod_num)
        index += 1

C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


 [index]:32628

C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


 [index]:199830

#### 搜索特征向量持久化

In [179]:
posseg_doc_vec.tofile('../../models/personas_posseg_sent_vec.text')

### 模型训练、筛选、评估与改进

#### 检查是否存在nan的向量

In [109]:
for index, i in enumerate(posseg_doc_vec):
    if sum(np.isnan(i)).sum():
        print(index)

32557
60085
87498
119773


In [178]:
empty_index = 0
def empty_list(obj):
    global empty_index
    index = empty_index
    if not obj:
        print(index)
    empty_index += 1
    return ' '.join(obj)
data_empty = data.PossegCleanedQueryList.apply(empty_list)

32557
60085
87498
119773


In [202]:
data[data.PossegCleanedQueryList=='']

,ID,QueryList,age,Gender,Education,PossegCleanedQueryList
32557,7077737F3F2D58E1EB0D3D2E8824B41D,pan.baidu.com/s/1esytqsi\tpan.baidu.com/s/1mhb...,2.0,1.0,3.0,
60085,5FAF74A9D9E2B55C6724BDEAA6D7574B,denied\tchimpanzee\tsanitation\tpredisposed\ti...,2.0,2.0,4.0,
87498,F723D21F70AA59932E0A77C12A8F4B95,substandard\twages\toutlay\tgovernance\tkeyboa...,3.0,1.0,4.0,
119773,CC302AED793BE054C6158160729F8687,550476145267\t550476291385\t666759024744\t5504...,NaN,NaN,NaN,


In [203]:
data[data.PossegCleanedQueryList==''].index.tolist()

[32557, 60085, 87498, 119773]

In [204]:
data_index = data.index.tolist()

In [206]:
for i in [32557, 60085, 87498, 119773]:
    data.drop(i, inplace=True)

In [207]:
data[data.PossegCleanedQueryList=='']

,ID,QueryList,age,Gender,Education,PossegCleanedQueryList


In [208]:
data.shape

(199827, 6)

#### 删除nan值的数据

In [209]:
nan_index_list = [32557,60085,87498,119773]
posseg_doc_vec[nan_index_list]

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [216]:
cleaned_posseg_doc_vec = np.delete(posseg_doc_vec,[32557,60085,87498,119773],axis=0)

In [218]:
cleaned_posseg_doc_vec.shape

(199827, 300)

#### 过滤为0的缺失值数据

In [223]:
data[0:99828].tail()

,ID,QueryList,age,Gender,Education,PossegCleanedQueryList
99826,E797FFCDCAF3899AB4D17B61170D8BFF,梦三生\t逆行天后漫画结局\t英雄联盟角色介绍大全\t韩剧网最新韩国电视剧\t119宣传语是...,1.0,1.0,5.0,梦 逆行 漫画 结局 英雄 联盟 角色 介绍 韩剧 网 韩国 电视剧 宣传语 斗破 苍穹 肆...
99827,E06375F7D092ABDE78C2D79E4725D6B0,中国军队配枪\t女生\t央视版权问题\t重庆\t一次性手术刀\t精神枷锁\t人民检察官\t舆...,1.0,1.0,5.0,中国 军队 配枪 女生 央视 版权 重庆 手术刀 精神枷锁 检察官 舆论 精神 禁锢 百度 ...
99828,D55119CB0B9366B20974522B58C00912,英文翻译\t幼儿园面试讲课\t学前教育书第二版\tshock to\t幼儿园教师资格证面试讲...,2.0,2.0,5.0,英文翻译 幼儿园 面试 讲课 书 版 幼儿园 教师 资格证 面试 讲课 设计 签名 免费 艺...
99829,EB4DBBD602C6459A19A77F09035E170C,哈尔滨祖研中医院地址\t指甲盖侧面的肉怎么是白色\t补骨质有副作用吗\t哈尔滨去呼兰\t黑龙...,3.0,2.0,3.0,哈尔滨 祖研 医院地址 指甲盖 肉 白色 补 骨质 副作用 哈尔滨 呼兰 黑龙江 祖研 中医...
99830,61CF81DB79423CB89E5DAA752BC4D9DD,陈翔毛晓彤分手\t青春偶像剧校园电影\t九尾狐家族\t王俊凯小说\t女娲的成长日记女娲变身\...,1.0,2.0,5.0,陈翔 分手 青春偶像 剧 校园 电影 九尾狐 家族 王俊凯 小说 女娲 成长 日记 女娲 变...


#### 训练集准备

In [225]:
import numpy as np
# 年龄
age_index = np.nonzero(data[0:99828].age)
# age_X = doc_vec[age_index]
posseg_age_X = cleaned_posseg_doc_vec[age_index]
age_y = data.age.iloc[age_index]

# 性别
gender_index = np.nonzero(data[0:99828].Gender)
# gender_X = doc_vec[gender_index]
posseg_gender_X = cleaned_posseg_doc_vec[gender_index]
gender_y = data.Gender.iloc[gender_index]

# 学历
education_index = np.nonzero(data[0:99828].Education)
# education_X = doc_vec[education_index]
posseg_education_X = cleaned_posseg_doc_vec[education_index]
education_y = data.Education.iloc[education_index]

In [80]:
print(age_X.shape,age_y.shape)

(98165, 300) (98165,)


In [82]:
print(education_X.shape,education_y.shape)

(90572, 300) (90572,)


In [227]:
print(gender_X.shape,gender_y.shape)

NameError: name 'gender_X' is not defined

In [229]:
print(posseg_age_X.shape,age_y.shape)

(98162, 300) (98162,)


In [230]:
print(posseg_gender_X.shape,gender_y.shape)

(97675, 300) (97675,)


In [231]:
print(posseg_education_X.shape,education_y.shape)

(90569, 300) (90569,)


##### 缺失值预测

In [83]:
from sklearn.model_selection import train_test_split
age_train_X, age_test_X, age_train_y, age_test_y = train_test_split(age_X, age_y, test_size=0.3,random_state=0)

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
models = [KNeighborsClassifier(),LogisticRegression(),GaussianNB(), SVC(),DecisionTreeClassifier(),RandomForestClassifier()]
for model in models:
    model.fit(age_train_X, age_train_y)
    print(model.score(age_test_X, age_test_y))
    

0.4499830220713073


C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5100509337860781
0.4049915110356537


C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.4993887945670628
0.359151103565365


C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.4360611205432937


In [234]:
from sklearn.model_selection import train_test_split
posseg_age_train_X, posseg_age_test_X, posseg_age_train_y, posseg_age_test_y = train_test_split(posseg_age_X, age_y, test_size=0.3,random_state=0)

In [236]:
# 经过词性过滤
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
models = [KNeighborsClassifier(),LogisticRegression(),GaussianNB(), SVC(),DecisionTreeClassifier(),RandomForestClassifier()]
for model in models:
    model.fit(posseg_age_train_X, posseg_age_train_y)
    print(model.score(posseg_age_test_X, posseg_age_test_y))

0.4496926890556555


C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5110530068932732
0.42361370504940743


C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.4935651465245
0.3592991273048321


C:\Users\xiaoyu\Envs\Machine_learning\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.44086386634520697


#### 模型筛选与改进

##### KNN

In [35]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
n_neighbors = np.arange(5,50,5)
weights  = ['distance', 'uniform']
p = [1, 2]
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
param_grid = {'n_neighbors':n_neighbors, 'weights':weights, 'p':p, 'algorithm':algorithm}
knn_gs = GridSearchCV(estimator=KNeighborsClassifier(n_jobs=4),param_grid=param_grid,verbose=True,cv=5)
knn_gs.fit(posseg_age_train_X[:10000], posseg_age_train_y[:10000])
print(knn_gs.best_score_)
print(svm_gs.best_estimator_)

NameError: name 'posseg_age_train_X' is not defined